In [1]:
#coding: utf-8
# 'my_breakout_a3c_20180419'
# A3CアルゴリズムによるOpen AI Gym自作ブロック崩し環境下での強化学習
# Ubuntuで走らせること。Windowsではエラーになる
# Pygameがマルチスレッドに対応していないため、PROCESSES=1では問題なくランするが、
# PROCESSES>1にすると終了してしまう（エラーになる）。

import os
print(os.getcwd())
os.environ['OMP_NUM_THREADS']='1'               # Numpyが複数スレッドを使用するのを防止

import chainer
import chainer.functions as F
import chainer.links as L
import gym
gym.undo_logger_setup()                         # Turn off gym's default logger settings
import gym.wrappers
import numpy as np
import sys
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')

import chainerrl
from chainerrl.agents import a3c
from chainerrl import experiments
from chainerrl import links
from chainerrl import misc
from chainerrl.optimizers.nonbias_weight_decay import NonbiasWeightDecay
from chainerrl.optimizers import rmsprop_async
from chainerrl import policies
from chainerrl.recurrent import RecurrentChainMixin
from chainerrl import v_function

import matplotlib
import matplotlib.pyplot as plt

import myenv                                    # 自作環境のインポート


PROCESSES = 2                                   # スレッド数（プロセス数）
# 環境変数（※注意！：環境と一致させる必要がある）
BANDWIDTH = 4                       # ニューラルネットへの入力チャンネル数
N_ACTIONS = 4                       # 行動の種類数（0:左移動,1:静止,2:右移動,3:ボール射出）


def phi(obs):
    return obs.astype(np.float32)


class QFunction(chainer.Chain):    
    def __init__(self, out_size):
        super().__init__(
            L0=L.Convolution2D(BANDWIDTH, 32, ksize=8, stride=4),
            L1=L.Convolution2D(32, 64, ksize=4, stride=2),
            L2=L.Convolution2D(64, 64, ksize=3, stride=1),
            L3=L.Linear(3136, 512),
            L4=L.Linear(512, out_size))
 
    def __call__(self, x, test=False):
        h = F.relu(self.L0(x))
        h = F.relu(self.L1(h))
        h = F.relu(self.L2(h))
        h = F.relu(self.L3(h))
        h = F.tanh(self.L4(h))
        return h


class My_A3CFFSoftmax(chainer.ChainList, a3c.A3CModel):
    """My example of A3C feedforward softmax policy."""

    def __init__(self):
        self.pi = policies.SoftmaxPolicy(
            model=QFunction(N_ACTIONS))
        self.v = QFunction(1)
        super().__init__(self.pi, self.v)

    def pi_and_v(self, state):
        return self.pi(state), self.v(state)


misc.set_random_seed(0)                         # Environment threadsで使用する乱数系列の設定
process_seeds = np.arange(PROCESSES)            # スレッド数分の乱数シードを用意
print('process_seeds', process_seeds)


def make_env(process_idx, test):
    env = gym.make('myenv-v0')
    # 当該プロセスで使用するシードを選択しenvにセット
    process_seed = int(process_seeds[process_idx])
    env_seed = 2**32-1- process_seed if test else process_seed
    env.seed(env_seed)
    if not test:
        reward_scale_factor = 1e-2
        misc.env_modifiers.make_reward_filtered(env, lambda x: x * reward_scale_factor)
    return env

#sample_env = gym.make('myenv-v0')
for j in range(PROCESSES):
    sample_env = make_env(process_seeds[j], False)
    sample_env.seed(j)
    obs_space = sample_env.observation_space
    action_space = sample_env.action_space

    print('Agent = ', j)
    print('observation space:', obs_space)
    print('action space:', action_space)

    obs = sample_env.reset()
    sample_env.render()
    print('initial observation:', obs)

    action = action_space.sample()
    obs, r, done, info = sample_env.step(action)
    print('next observation:', obs)
    print('reward:', r)
    print('done:', done)
    print('info:', info)
    print('action;', action)

    print('obs.shape;', obs.shape)
    print('obs[0].shape;',obs[0].shape)
    for i in range(BANDWIDTH):
        plt.figure()
        plt.imshow(obs[i])
    print(matplotlib.get_backend())


# エージェント・モデル（ニューラルネットの設定）
model = My_A3CFFSoftmax()

# 最適化の設定（たぶんA3Cの論文にあるアルゴリズム）
opt = rmsprop_async.RMSpropAsync(lr=7e-4, eps=0.1, alpha=0.99)  
opt.setup(model)
opt.add_hook(chainer.optimizer.GradientClipping(40))        # |勾配|を40以下に抑える

# Now create an agent that will interact with the environment.
agent = a3c.A3C(model, opt, t_max=5, gamma=0.99, beta=1e-2, phi=phi)


/home/toshi/My_Breakout_A3C


/home/toshi/anaconda3/envs/Chainer30python36/lib/python3.6/site-packages/gym/__init__.py:15: UserWarning: gym.undo_logger_setup is deprecated. gym no longer modifies the global logging configuration
  warnings.warn("gym.undo_logger_setup is deprecated. gym no longer modifies the global logging configuration")


process_seeds [0 1]
/home/toshi/My_Breakout_A3C
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
Agent =  0
observation space: Box(400, 400, 3)
action space: Discrete(4)
initial observation: [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
next observation: [[[11 11 11 ... 11 11 11]
  [11 11 11 ... 11 11 11]
  [11 11 11 ... 11 11 11]
  ...
  [11 11 11 ... 11 11 11]
  [11 11 11 ... 11 11 11]
  [11 11 11 ... 11 11 11]]

 [[ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...  0  0  0]
  [ 0  0  0 ...

In [14]:
#agent.load('myenv1000000')                       # 学習済みエージェントを使用するとき

import time                                     # 時間計測の設定
start = time.time()

experiments.train_agent_async(                  # 学習訓練と評価
    agent = agent,                              # 学習エージェント
    outdir = 'result',                          # 結果出力ファイルの指定
    processes = PROCESSES,                      # スレッド数（プロセス数）
    make_env = make_env,                        # 環境
    profile = False,                            # 
    steps = 1000000,                            # 学習ステップ数
                                                #（エピソードとは異なるようだが、このステップ数で終了になる）
    eval_n_runs = 10,                           # 評価時の統計回数
    eval_interval = 50000,                      # 評価のインターバル・ステップ
    max_episode_len = 200)                      # エピソードの長さの最大値（ここでシミュレーションを打ちきる）

agent.save('myenv1000000')                       # 学習エージェントの保存（'result'にも保存されているようだ）

print('Finished, elapsed time : {} sec'.format(time.time()-start))      # 処理にかかった時間の表示

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.uint8'>. Please provide explicit dtype.
Finished, elapsed time : 5.645029783248901 sec


In [3]:
agent.save('myenv1000000') 

In [9]:
print(os.getcwd())

/home/toshi/myenv_mya3c
